<a href="https://colab.research.google.com/github/hungryphobic/NLP-projects/blob/main/P3_TwitterSentimentAnalysisMalayLanguage_FeatureExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Feature Extraction**
---

## **Introduction**

This phase is we take the cleaned dataset and performed feature extraction so that it can be use for classificaion task later on. Feature extraction which also known as vectorization; vectorization the general process of turning a collection of text documents into numerical feature vectors. Because it prepare the data in such it can be use; as sequence of symbols cannot be fed directly to the algorithms themselves as most of them expect numerical feature vectors with a fixed size rather than the raw text documents with variable length.

After some digging, there is some common vectorization method, technique wehich is bag if Word(BOW),count vectorizer and tfidf vectorizer. in this project I will compare the performance of count vectorizer and tfidf vectorizer; and mainly focus on tfidf as from my reading, **The hypothesis** is that tfidf vectorizer is better in term of performance and classification task later on. So,let's find out!


For this part of project i will use scikit library. As scikit-learn provides utilities for the most common ways to extract numerical features from text content, namely:

> * **tokenizing** strings and giving an integer id for each possible token, for instance by using white-spaces and punctuation as token separators.
* **counting** the occurrences of tokens in each document.
* **normalizing** and weighting with diminishing importance tokens that occur in the majority of samples / documents.

---

# Import library package and module

In [1]:
#import module and package
%%time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

CPU times: user 332 ms, sys: 162 ms, total: 494 ms
Wall time: 719 ms


In [ ]:
# just to try the vectorizer
texts = ["good movie","not a good movie","did not like","i like it","good one"]

## **Pre-V**

Just do simple vectorization for simple text data just to do an overview of both vectorizer
---

### **Count vectorizer**
---
> CountVectorizer implements both tokenization and occurrence counting in a single class

#### Initilize

In [ ]:
cV = CountVectorizer(ngram_range=(1, 2),min_df=2) 

#### vectorize our corpus

In [ ]:
x = cV.fit_transform(texts)

#### View or vectorized data

In [ ]:
print(x.toarray())

[[1 1 0 1 0]
 [1 1 0 1 1]
 [0 0 1 0 1]
 [0 0 1 0 0]
 [1 0 0 0 0]]


In [ ]:
pd.DataFrame(x.todense(), columns=cV.get_feature_names())

,good,good movie,like,movie,not
0,1,1,0,1,0
1,1,1,0,1,1
2,0,0,1,0,1
3,0,0,1,0,0
4,1,0,0,0,0


### Tf idf Vectorizer
---
> **Tf** means ***term-frequency*** while **tf–idf**   means term-frequency times ***inverse document-frequency***: 

> `tf-idf(t,d) = tf(t,d) x idf(t)`

In a large text corpus, some words will be very present (e.g. “the”, “a”, “is” in English) hence carrying very little meaningful information about the actual contents of the document. If we were to feed the direct count data directly to a classifier those very frequent terms would shadow the frequencies of rarer yet more interesting terms.

>In order to re-weight the count features into floating point values suitable for usage by a classifier it is very common to use the **tf–idf transform**.

#### Initialize

In [ ]:
tV = TfidfVectorizer(min_df=2, ngram_range=(1,2))

#### Vectorize our Corpus

In [ ]:
y = tV.fit_transform(texts)

#### View our vectorized data

In [ ]:
print(y.toarray())

[[0.50620441 0.60981846 0.         0.60981846 0.        ]
 [0.43218341 0.52064623 0.         0.52064623 0.52064623]
 [0.         0.         0.70710678 0.         0.70710678]
 [0.         0.         1.         0.         0.        ]
 [1.         0.         0.         0.         0.        ]]


In [ ]:
pd.DataFrame(y.todense(), columns=tV.get_feature_names())

,good,good movie,like,movie,not
0,0.506204,0.609818,0.000000,0.609818,0.000000
1,0.432183,0.520646,0.000000,0.520646,0.520646
2,0.000000,0.000000,0.707107,0.000000,0.707107
3,0.000000,0.000000,1.000000,0.000000,0.000000
4,1.000000,0.000000,0.000000,0.000000,0.000000


## Vectorization
---

### V-1)Tf-idf 
  multipleFileTfidfVectorizer(min_file,max_file,Vectorizer) 
---
1. step1: load cleaned data(contain nan value) --> data_i
2. step2: remove nan value datase
3. step 3: saved the non empty cleaned data
4. step 4: load again our non empty data and assign..
  *  I) data_i_Text 
  *  II) data_i_Polarity
5. step 5: vectorize our text data --> data_i_Vec
6. step 6: merge our data <-- data_i_Vec concat data_i_Polarity
7. step 7: Export our dataset
   
* ***fp_1*** is file path to import cleaned datset from cleaned dataset folder (refer step 1)
* ***fp_2*** is file path to export nan removed dataset to nanRemove folder. Also th epath where the datset will be import (refer step and step4)
* ***fp_3*** is file path to saved our final dataset (refer step 7)


In [2]:
def multipleFileTfidfVectorizer(min_file,max_file, vector):  
  fp_1 = '/content/drive/MyDrive/Colab Notebooks/tweets_data/for feature extraction/clean dataset/'
  fp_2 = '/content/drive/MyDrive/Colab Notebooks/tweets_data/for feature extraction/clean dataset/nanRemove/'
  fp_3 = '/content/drive/MyDrive/Colab Notebooks/tweets_data/for feature extraction/clean dataset/vectorized/tfidf/'
  
  for i in range(min_file,max_file+1):
    #initialized our variable
    raw = 'data'+str(i)
    fp1 = fp_1+str(i)+'.csv'
    fp2 = fp_2+str(i)+'.csv'
    fp3 = fp_3+str(i)+'.csv'
    #step1
    raw = pd.read_csv(fp1)
    #step2
    data = raw.dropna()
    print('========================================')
    print('Dataset '+str(i)+' details :')
    data.info()
    #step3
    data.to_csv(fp2, index=False)
    #step4
    data2 = pd.read_csv(fp2)
    data2Text= data2['CD-RE']
    data2Text= list(data2Text)
    data2Polarity = data2['Polarity']
    #step5
    data2Vector = vector.fit_transform(data2Text)
    #step6
    data2Vec = pd.DataFrame(data2Vector.todense(), columns=vector.get_feature_names())
      # merge our vectorized text data wuth their labelled data
    data2ToProcess = pd.concat([data2Vec, data2Polarity], axis=1, ignore_index=False)
    #step7
    data2ToProcess.to_csv(fp3, index=False)
    # %%time
    print('Dataset '+str(i)+' finished successfully!!')
    print('========================================')

In [3]:
%%time
tfidf= TfidfVectorizer(min_df=2, ngram_range=(1,2))
multipleFileTfidfVectorizer(19,19,tfidf)

Dataset 19 details :
<class 'pandas.core.frame.DataFrame'>
Int64Index: 683 entries, 0 to 791
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   CD-RE     683 non-null    object 
 1   Polarity  683 non-null    float64
dtypes: float64(1), object(1)
memory usage: 16.0+ KB
Dataset 19 finished successfully!!
CPU times: user 841 ms, sys: 96.2 ms, total: 937 ms
Wall time: 2.13 s


### V-2)Count Vectorizer
--- 

In [5]:
def multipleFileCountVectorizer(min_file,max_file, vector):  
  
  fp_11 = '/content/drive/MyDrive/Colab Notebooks/tweets_data/for feature extraction/clean dataset/nanRemove/'
  fp_12 = '/content/drive/MyDrive/Colab Notebooks/tweets_data/for feature extraction/clean dataset/vectorized/count vectorizer/'
  
  for i in range(min_file,max_file+1):
    #initialized our variable
    fp1 = fp_11+str(i)+'.csv'
    fp2 = fp_12+str(i)+'.csv'
    
    #step1
    data = pd.read_csv(fp1)
    dataText= data['CD-RE']
    dataText= list(dataText)
    dataPolarity = data['Polarity']
    #step2
    dataVector = vector.fit_transform(dataText)
    #step3
    dataVec = pd.DataFrame(dataVector.todense(), columns=vector.get_feature_names())
      # merge our vectorized text data wuth their labelled data
    dataToProcess = pd.concat([dataVec, dataPolarity], axis=1, ignore_index=False)
    #step4
    dataToProcess.to_csv(fp2, index= False)
    
    print('Dataset '+str(i)+' finished successfully!!')
    print('========================================')

In [6]:
%%time
countVectorizer = CountVectorizer(ngram_range=(1, 2),min_df=2) 
multipleFileCountVectorizer(19,19,countVectorizer)

Dataset 19 finished successfully!!
CPU times: user 170 ms, sys: 2.15 ms, total: 172 ms
Wall time: 387 ms
